In [ ]:
# I am stopping work on this collab and starting work on my time series models
# -Vincent

In [ ]:
# https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/
# mount Google Drive so we can access it
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# file paths to our data
!ls 'gdrive/Shareddrives/CPSC 4176 Project Spring 2023/UAV Data'

In [ ]:
from tensorflow import keras
import pandas as pd

In [ ]:
url = "gdrive/Shareddrives/CPSC 4176 Project Spring 2023/UAV Data/UAV_4_Data_down.xlsx"
data = pd.read_excel(url)

In [ ]:
data.shape

(472241, 10)

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data.transpose()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# displays all vehicles speed and times for Lane 1
data.plot(kind='line', x='Time(s)', y='Speed(m/s)', color='blue', title="All Vehicles Lane 1")

In [ ]:
# counts and displays how many datapoints per vehicle
vehicle_datapoints = {}
for vehicle in data['VehicleID']:
  vehicle_datapoints[vehicle] = vehicle_datapoints.get(vehicle, 0) + 1
vehicle_datapoints

In [ ]:
# searches for specific VehicleID and extracts all data for that vehicle
search_vehicle_id = 13909
vehicle_data = data.loc[data['VehicleID'] == search_vehicle_id]
vehicle_data

,VehicleID,LaneID,Time(s),x-axis position(m),Distance from upper Lane line(m),Distance from lower lane line(m),Speed(m/s),Acceleration(m/s^2),VehicleLength(meter),VehicleWidth(meter)
433090,13909,1,1097.82354,3.230000,0.666967,2.523576,12.508598,-0.067542,4.168776,1.965493
433091,13909,1,1097.85687,3.646912,0.665265,2.528267,12.506349,-0.065788,4.168776,1.965493
433092,13909,1,1097.89020,4.063748,0.663626,2.532895,12.504158,-0.064101,4.168776,1.965493
433093,13909,1,1097.92353,4.480512,0.662043,2.537465,12.502024,-0.070217,4.168776,1.965493
433094,13909,1,1097.95686,4.897204,0.660512,2.541983,12.499686,-0.060820,4.168776,1.965493
...,...,...,...,...,...,...,...,...,...,...
433893,13909,1,1124.58753,352.199367,0.891840,2.682789,15.645186,0.102855,4.168776,1.965493
433894,13909,1,1124.62086,352.720821,0.892291,2.683563,15.648611,0.105459,4.168776,1.965493
433895,13909,1,1124.65419,353.242389,0.892797,2.684284,15.652123,0.116001,4.168776,1.965493
433896,13909,1,1124.68752,353.764074,0.893283,2.685024,15.655985,0.087254,4.168776,1.965493


In [ ]:
# plots individual vehicle speed, acceleration, lane position
vehicle_data.plot(kind='line', x='Time(s)', y='Speed(m/s)', color='blue', title="Vehicle: " + str(search_vehicle_id))
vehicle_data.plot(kind='line', x='Time(s)', y='Acceleration(m/s^2)', color='blue', title="Vehicle: " + str(search_vehicle_id))
vehicle_data.plot(kind='line', x='Time(s)', y='Distance from upper Lane line(m)', color='blue', title="Vehicle: " + str(search_vehicle_id))
vehicle_data.plot(kind='line', x='Time(s)', y='Distance from lower lane line(m)', color='blue', title="Vehicle: " + str(search_vehicle_id))

In [ ]:
# removed feature values to keep initial model simple
vehicle_data = vehicle_data.drop(labels='LaneID', axis=1)
vehicle_data = vehicle_data.drop(labels='Distance from upper Lane line(m)', axis=1)
vehicle_data = vehicle_data.drop(labels='Distance from lower lane line(m)', axis=1)
vehicle_data = vehicle_data.drop(labels='VehicleLength(meter)', axis=1)
vehicle_data = vehicle_data.drop(labels='VehicleWidth(meter)', axis=1)
vehicle_data = vehicle_data.drop(labels='x-axis position(m)', axis=1)
vehicle_data = vehicle_data.drop(labels='VehicleID', axis=1)
vehicle_data = vehicle_data.reset_index()
vehicle_data = vehicle_data.drop(labels='index', axis=1)
vehicle_data

,Time(s),Speed(m/s),Acceleration(m/s^2)
0,1097.82354,12.508598,-0.067542
1,1097.85687,12.506349,-0.065788
2,1097.89020,12.504158,-0.064101
3,1097.92353,12.502024,-0.070217
4,1097.95686,12.499686,-0.060820
...,...,...,...
803,1124.58753,15.645186,0.102855
804,1124.62086,15.648611,0.105459
805,1124.65419,15.652123,0.116001
806,1124.68752,15.655985,0.087254


In [ ]:
# We need to normalize the data between 0 and 1
# data' is normalized between 0 and 1
# data' = (data - data_min) / (data_max - data_min)

def NormalizeData(data):
  return (data - np.min(data)) / (np.max(data) - np.min(data))
def NormalizeData2(data):
  return (data - data.mean(axis=0)) / data.std(axis=0)

normalize_data = pd.DataFrame()
normalize_data['Time'] = vehicle_data['Time(s)']
normalize_data['Speed'] = NormalizeData(vehicle_data['Speed(m/s)'])
normalize_data['Acceleration'] = NormalizeData(vehicle_data['Acceleration(m/s^2)'])

normalize_data2 = pd.DataFrame()
normalize_data2['Time'] = vehicle_data['Time(s)']
normalize_data2['Speed'] = NormalizeData2(vehicle_data['Speed(m/s)'])
normalize_data2['Acceleration'] = NormalizeData2(vehicle_data['Acceleration(m/s^2)'])

In [ ]:
normalize_data

,Time,Speed,Acceleration
0,1097.82354,0.307086,0.264639
1,1097.85687,0.306592,0.266991
2,1097.89020,0.306110,0.269252
3,1097.92353,0.305640,0.261053
4,1097.95686,0.305126,0.273652
...,...,...,...
803,1124.58753,0.996985,0.493087
804,1124.62086,0.997739,0.496578
805,1124.65419,0.998511,0.510712
806,1124.68752,0.999361,0.472171


In [ ]:
print(NormalizeData2(clean_file))

In [ ]:
# display min/max normalization
print("Max Speed: ", vehicle_data['Speed(m/s)'].max(), "Min Speed: ", vehicle_data['Speed(m/s)'].min())
print("Max Accel: ", vehicle_data['Acceleration(m/s^2)'].max(),"Min Accel: ",  vehicle_data['Acceleration(m/s^2)'].min())
print("Max Speed: ", normalize_data['Speed'].max(), "Min Speed: ", normalize_data['Speed'].min())
print("Max Accel: ", normalize_data['Acceleration'].max(),"Min Accel: ",  normalize_data['Acceleration'].min())
normalize_data

Max Speed:  15.658891018909252 Min Speed:  11.112446958988873
Max Accel:  0.48095757666038097 Min Accel:  -0.26493425834897844
Max Speed:  1.0 Min Speed:  0.0
Max Accel:  1.0 Min Accel:  0.0


,Time,Speed,Acceleration
0,1097.82354,0.307086,0.264639
1,1097.85687,0.306592,0.266991
2,1097.89020,0.306110,0.269252
3,1097.92353,0.305640,0.261053
4,1097.95686,0.305126,0.273652
...,...,...,...
803,1124.58753,0.996985,0.493087
804,1124.62086,0.997739,0.496578
805,1124.65419,0.998511,0.510712
806,1124.68752,0.999361,0.472171


In [ ]:
# display mean/standard deviation normalization
# this doesn't scale to 0 to 1, unsure of why
print("Max Speed: ", vehicle_data['Speed(m/s)'].max(), "Min Speed: ", vehicle_data['Speed(m/s)'].min())
print("Max Accel: ", vehicle_data['Acceleration(m/s^2)'].max(),"Min Accel: ",  vehicle_data['Acceleration(m/s^2)'].min())
print("Max Speed: ", normalize_data2['Speed'].max(), "Min Speed: ", normalize_data2['Speed'].min())
print("Max Accel: ", normalize_data2['Acceleration'].max(),"Min Accel: ",  normalize_data2['Acceleration'].min())
normalize_data2

Max Speed:  15.658891018909252 Min Speed:  11.112446958988873
Max Accel:  0.48095757666038097 Min Accel:  -0.26493425834897844
Max Speed:  1.7667936346080502 Min Speed:  -1.3179871512857533
Max Accel:  1.6598387241999657 Min Accel:  -1.7439176660957867


,Time,Speed,Acceleration
0,1097.82354,-0.370693,-0.843151
1,1097.85687,-0.372219,-0.835145
2,1097.89020,-0.373705,-0.827448
3,1097.92353,-0.375154,-0.855358
4,1097.95686,-0.376740,-0.812473
...,...,...,...
803,1124.58753,1.757495,-0.065571
804,1124.62086,1.759818,-0.053688
805,1124.65419,1.762201,-0.005580
806,1124.68752,1.764822,-0.136764


In [ ]:
# TODO: Now that data is normalized, make it into a RNN model
# Need to split into train, val, test
# this doesn't randomly 
df_size = len(vehicle_data)
train_size = 0.7
val_size = 0.2
test_size = 0.1

train_df = vehicle_data[0:int(df_size * 0.7)]
val_df = vehicle_data[int(df_size * 0.7):int(df_size * 0.9)]
test_df = vehicle_data[int(df_size * 0.9):]
train_df

,Time(s),Speed(m/s),Acceleration(m/s^2)
0,1097.82354,12.508598,-0.067542
1,1097.85687,12.506349,-0.065788
2,1097.89020,12.504158,-0.064101
3,1097.92353,12.502024,-0.070217
4,1097.95686,12.499686,-0.060820
...,...,...,...
560,1116.48834,14.171889,0.171188
561,1116.52167,14.177589,0.163036
562,1116.55500,14.183018,0.158961
563,1116.58833,14.188312,0.130429


In [ ]:
# just examples of how to build a model 
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

In [ ]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary()


In [ ]:
# ---------------------------------------------------------------- DATA CLEANING SECTION [HIMANSHU BOHRA, DONGHYEON PARK] ---------------------------------------- #

# DATA CLEANING/PREP SECTION [ PRE-NORMALIZATION]

clean_file = data.copy()

# Cleaning function
def clean_speed_accel_UAV4(inputData):
  i=0
  timeConstant = 0.03333
  old_data = None
  errors_found = 0

  while(i<inputData.shape[0]-1):
    # Cleaning the speed in UAV4
    old_data = inputData["Speed(m/s)"][inputData.index[i]] # Record old data
    inputData["Speed(m/s)"][inputData.index[i]] = (inputData["x-axis position(m)"][inputData.index[i+1]] - inputData["x-axis position(m)"][inputData.index[i]]) / timeConstant
    # Add to the error count if old_data was changed
    if(old_data != inputData["Speed(m/s)"][inputData.index[i]]): errors_found = errors_found+1
    # Cleaning the accel in UAV4
    old_data = inputData["Acceleration(m/s^2)"][inputData.index[i]]
    inputData["Acceleration(m/s^2)"][inputData.index[i]] = (inputData["Speed(m/s)"][inputData.index[i+1]] -inputData["Speed(m/s)"][inputData.index[i]]) / timeConstant
    # Same for error count:
    if(old_data != inputData["Acceleration(m/s^2)"][inputData.index[i]]): errors_found = errors_found+1
    i = i+1
    return [inputData, errors_found]


# Sorting, cleaning and concatenating by VehicleIDs to prevent speed/acc miscalculation.
def sort_by_VID(inputData):
    vID_list = inputData["VehicleID"].unique()

    # Resultant array containing ALL the cleaned dataframes for individual vehicle ID rows:
    cleaned_vehicle_rows = []
    
    # Iterators:
    vid_iterator = 0

    # NESTED LOOP VARIABLES FOR ERROR COUNT
    total_errors_found = 0
    tempArray = [None, None] # Result storage to reduce processing load [clean function is expensive]

    while(vid_iterator < len(vID_list)):
      currentVehicleIDRows = data.loc[data['VehicleID'] == vID_list[vid_iterator]]

      tempArray = clean_speed_accel_UAV4(currentVehicleIDRows)
      cleaned_vehicle_rows.append(tempArray[0])
      total_errors_found = total_errors_found + tempArray[1]

      vid_iterator = vid_iterator + 1
      # END OF OUTER LOOP

    return [pd.concat( cleaned_vehicle_rows ) , total_errors_found ]


# Applying the function to clean the data
temp_result_array = sort_by_VID(clean_file)
clean_file = temp_result_array[0]
total_errors_from_cleaning = temp_result_array[1]



'''
normalized_file = pd.DataFrame()
normalized_file['VehicleID'] = data['VehicleID']
normalized_file['LaneID'] = data['LaneID']
normalized_file['Time(s)'] = data['Time(s)
normalized_file['x-axis position(m)'] = NormalizeData(data['x-axis position(m)'])
normalized_file['VehicleID'] = data['VehicleID']
normalized_file['VehicleID'] = data['VehicleID']']
'''

#clean_file["Acceleration(m/s^2)"]
print(clean_file[["Speed(m/s)","Acceleration(m/s^2)"]])

In [ ]:
# TEST AREA

#clean_file["Speed(m/s)"][clean_file.index[472239]]
#clean_file.loc[clean_file["Speed(m/s)"] == 12.852531812619823 ]
print(total_errors_from_cleaning)

In [ ]:
# Saving to drive, to reduce processing load re-occurance.
#clean_file.to_excel(r'gdrive/Shareddrives/CPSC 4176 Project Spring 2023/Code/NormalizedDataExcelFile/UAV_4_Data_down_Cleaned.xlsx', index=False)

# Logging the errors from the above code block's cleaning:
with open('gdrive/Shareddrives/CPSC 4176 Project Spring 2023/Code/NormalizedDataExcelFile/UAV_4_Data_down_ERRORLOG.txt', 'w') as f:
  f.write('Total record check: ' + str(clean_file.shape[0]) + "\n")
  f.write('Total Errors Found: ' + str(total_errors_from_cleaning))


# ----------------------------------------------------------------------- END OF DATA CLEANING ----------------------------------------------------------------#
